# Bank Customer Churn Prediction - Data Preparation and Modeling

This notebook focuses on preparing the data for modeling and building predictive models to identify customers likely to churn. We will:

1. Load and prepare the data
2. Engineer features 
3. Split the data into training and testing sets
4. Build and evaluate multiple models
5. Interpret the results
6. Draw conclusions and make recommendations

Let's begin by loading our dataset and the necessary libraries.

In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import shap
import warnings

# Configure visualizations
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
warnings.filterwarnings('ignore')

ImportError: Numba needs NumPy 1.24 or greater. Got NumPy 1.23.